In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [5]:
from app.resume_parser import extract_text_from_resume,extract_resume_sections
from app.linkedin_scraper import scrape_jd_from_url

In [12]:
def scrape_jobs_for_keywords(job_keywords, location="India", scroll_page=5):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-notifications")
    # options.add_argument("--headless")  # optional: remove this line if you want to see the browser
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    all_jobs_df=pd.DataFrame()
    for keyword in job_keywords:
        print(f"\n🔍 Searching for: {keyword}")
        query = keyword.replace(" ", "%20")
        loc = location.replace(" ", "%20")
        url = f"https://www.linkedin.com/jobs/search/?keywords={query}&location={loc}&f_TPR=r86400"  # last 24h jobs
        # url = f"https://www.linkedin.com/jobs/search/?keywords={query}&location={loc}" 
        driver.get(url)
        time.sleep(4)
        ActionChains(driver).move_by_offset(50, 50).click().perform()
        
        for _ in range(scroll_page):  # You can increase the range for more scrolls
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
        job_list = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
        job_cards = job_list.find_elements(By.TAG_NAME, "li")
        print(f"Total job cards found for keyword {keyword} are:{len(job_cards)}")
        job_urls = []
        jobs = []
        for card in job_cards:
            try:
                title =card.find_element(By.CLASS_NAME, "base-search-card__title").text.strip()
                company = card.find_element(By.CLASS_NAME, "base-search-card__subtitle").find_element(By.TAG_NAME, "a").text.strip()
                link =card.find_element(By.CLASS_NAME, "base-card__full-link").get_attribute("href")
                jobs.append({"title": title, "company": company, "link": link})
            
                #the below code also works for extracting links
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href")
                job_urls.append(link)
            except:
                pass 
        df=pd.DataFrame(jobs)
        df["keyword"]=keyword
        all_jobs_df = pd.concat([all_jobs_df, df], ignore_index=True)

    driver.quit()
    return all_jobs_df

In [13]:
keywords = [
    "LLM Engineer",
    "Generative AI Engineer"
]
all_jobs_df=scrape_jobs_for_keywords(job_keywords=keywords, location="Bengaluru", scroll_page=1)


🔍 Searching for: LLM Engineer
Total job cards found for keyword LLM Engineer are:8

🔍 Searching for: Generative AI Engineer
Total job cards found for keyword Generative AI Engineer are:12


In [14]:
all_jobs_df

,title,company,link,keyword
0,Senior LLM Engineer (With ML and Prompt Engg Exp),SciSpace,https://in.linkedin.com/jobs/view/senior-llm-e...,LLM Engineer
1,Senior LLM Engineer,SciSpace,https://in.linkedin.com/jobs/view/senior-llm-e...,LLM Engineer
2,AI / ML Engineer,Yellow.ai,https://in.linkedin.com/jobs/view/ai-ml-engine...,LLM Engineer
3,AI Engineer - LLM Applications & Data Mining,Expertia AI,https://in.linkedin.com/jobs/view/ai-engineer-...,LLM Engineer
4,Senior ML Ops Engineer,StockX,https://in.linkedin.com/jobs/view/senior-ml-op...,LLM Engineer
5,Senior AI Engineer - LLM Apps & Data Mining,Expertia AI,https://in.linkedin.com/jobs/view/senior-ai-en...,LLM Engineer
6,SAP BTP LLM DEVELOPER-Senior,EY,https://in.linkedin.com/jobs/view/sap-btp-llm-...,LLM Engineer
7,Senior Computer Vision Engineer,Maximl,https://in.linkedin.com/jobs/view/senior-compu...,LLM Engineer
8,AI / ML Engineer,Yellow.ai,https://in.linkedin.com/jobs/view/ai-ml-engine...,Generative AI Engineer
9,AI ML Engineer,Lifesight,https://in.linkedin.com/jobs/view/ai-ml-engine...,Generative AI Engineer


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-notifications")
# options.add_argument("--headless")  # optional: remove this line if you want to see the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
all_jobs = []

keyword="NLP"
location="Bengaluru"
print(f"\n🔍 Searching for: {keyword}")
query = keyword.replace(" ", "%20")
loc = location.replace(" ", "%20")
url = f"https://www.linkedin.com/jobs/search/?keywords={query}&location={loc}&f_TPR=r86400"  # last 24h jobs
# url = f"https://www.linkedin.com/jobs/search/?keywords={query}&location={loc}" 
driver.get(url)

time.sleep(5)
ActionChains(driver).move_by_offset(50, 50).click().perform()

for _ in range(1):  # You can increase the range for more scrolls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 
    
job_list = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
job_cards = job_list.find_elements(By.TAG_NAME, "li")
print(f"Total job cards found for keyword {keyword} are:{len(job_cards)}")
job_urls = []
jobs = []
for card in job_cards:
    try:
        title =card.find_element(By.CLASS_NAME, "base-search-card__title").text.strip()
        company = card.find_element(By.CLASS_NAME, "base-search-card__subtitle").find_element(By.TAG_NAME, "a").text.strip()
        link =card.find_element(By.CLASS_NAME, "base-card__full-link").get_attribute("href")
        jobs.append({"title": title, "company": company, "link": link})

        #the below code also works for extracting links
        link = card.find_element(By.TAG_NAME, "a").get_attribute("href")
        job_urls.append(link)
    except:
        pass 
driver.quit()

df=pd.DataFrame(jobs)
df["keyword"]=keyword

In [ ]:
df